In [1]:
import os
import re
import json
import glob
import fitz
import shutil

import pandas as pd
from pathlib import Path

In [2]:
def remove_pages(input_path, output_path, pages_to_remove):
    try:
        # 임시 파일 경로 생성
        temp_output = output_path + ".temp"

        # 파일 열기
        doc = fitz.open(input_path)

        # 페이지 제거
        for page_num in sorted(pages_to_remove, reverse=True):
            doc.delete_page(page_num - 1)

        # 임시 파일로 저장
        doc.save(temp_output)
        doc.close()

        # 원본 파일이 있으면 삭제 시도
        if os.path.exists(output_path):
            try:
                os.remove(output_path)
            except PermissionError:
                print(
                    f"경고: '{output_path}' 파일을 삭제할 수 없습니다. 다른 이름으로 저장합니다."
                )
                output_path = output_path.replace(".pdf", "_new.pdf")

        # 임시 파일을 최종 파일로 이동
        shutil.move(temp_output, output_path)
        return True, output_path

    except Exception as e:
        print(f"오류 발생: {input_path} 처리 중 - {str(e)}")
        return False, None


def process_pdf_files(input_dir, output_dir, other_file_names):
    # 출력 디렉토리가 없으면 생성
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # 처리 결과 추적
    success_count = 0
    error_count = 0

    # 입력 디렉토리의 모든 PDF 파일 처리
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(".pdf"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)

            # 파일명에서 확장자 제거
            file_name_without_ext = os.path.splitext(filename)[0]

            # other_file_names 목록에 있는 파일은 1-3페이지 제거
            if file_name_without_ext in other_file_names:
                pages_to_remove = [1, 2, 3]  # 1, 2, 3 페이지 제거
                print(f"1-3 페이지 제거 '{filename}'")
            else:
                # 그 외 파일은 1-2페이지만 제거
                pages_to_remove = [1, 2]  # 1, 2 페이지 제거
                print(f"1-2 페이지 제거 '{filename}'")

            success, final_path = remove_pages(input_path, output_path, pages_to_remove)
            if success:
                success_count += 1
            else:
                error_count += 1

    print(f"\n처리 완료: 성공 {success_count}개, 실패 {error_count}개")

In [3]:
def split_document_for_md(file_path, remove_figures=True):
    """
    마크다운 파일을 다양한 형식의 섹션으로 분할합니다:
    1. 숫자 패턴 (예: '1.', '2.', '1.1', '1.1.1')으로 시작하는 주요 섹션
    2. 괄호 숫자 패턴 (예: '(1)', '(2)')으로 시작하는 하위 섹션
    3. 괄호 한글 패턴 (예: '(가)', '(나)')으로 시작하는 더 하위 섹션

    각 섹션은 상위 섹션에 대한 참조 정보(high_level)를 포함합니다.

    Args:
        file_path (str): 분할할 마크다운 파일 경로
        remove_figures (bool): <그림> 태그가 있는 줄을 제거할지 여부 (기본값: True)

    Returns:
        list: 분할된 섹션 목록 (각 섹션은 딕셔너리 형태로 메타데이터 포함)

    최종 결과 형식:
    [
        {
            "type": "main|sub|subsub|intro",  # 섹션 유형
            "level": 0~5,                     # 섹션 레벨 (0:intro, 1:1., 2:1.1, 3:1.1.1, 4:(1), 5:(가))
            "number": "1.|1.1|(1)|(가)",      # 섹션 번호
            "title": "섹션 제목",              # 섹션 제목
            "content": "섹션 전체 내용",       # 섹션 내용
            "id": "1.|1.-1.1|1.-1.1-(1)|1.-1.1-(1)-(가)",  # 고유 식별자 (계층 구조를 '-'로 연결)
            "breadcrumb": "1. 제목 > 1.1 부제목 > (1) 항목 > (가) 세부항목", # 전체 경로
            "high_level": {                   # 상위 섹션 정보
                "level_1_number": "1.",
                "level_1_title": "상위 제목",
                "level_2_number": "1.1",
                "level_2_title": "상위 부제목"
                # 기타 상위 레벨 정보...
            },
            "file_path": "./path/to/file.md"  # 원본 파일 경로
        },
        # 기타 섹션들...
    ]

    JSONL 형식으로 저장 시:
    {
        "page_content": "섹션 전체 내용",
        "metadata": {
            "id": "1.-1.1-(1)-(가)",  # 모든 계층을 '-'로 연결한 식별자
            "type": "main|sub|subsub|intro",
            "number": "1.|1.1|(1)|(가)",
            "level": 0~5,
            "title": "섹션 제목",
            "breadcrumb": "1. 제목 > 1.1 부제목 > (1) 항목 > (가) 세부항목",
            "high_level": { ... },
            "file_path": "./path/to/file.md",
            "file_name": "파일명"
        }
    }
    """
    # 파일 내용 읽기
    with open(file_path, "r", encoding="utf-8-sig") as f:
        content = f.read()

    # <그림> 태그 제거 옵션이 활성화된 경우
    if remove_figures:
        figure_pattern = r"^\s*<그림\s+\d+(?:\.\d+)*>\s+.*$"
        lines = content.split("\n")
        filtered_lines = [
            line for line in lines if not re.match(figure_pattern, line.strip())
        ]
        content = "\n".join(filtered_lines)

    # 주요 섹션 패턴 (1., 1.1, 1.1.1 형식)
    main_pattern = r"^((?:[1-9]\d*\.)+(?:[1-9]\d*)?)\s+([A-Z가-힣].*?)$"

    # 하위 섹션 패턴 ((1), (2) 형식)
    sub_pattern = r"^\((\d+)\)\s+([A-Z가-힣].*?)$"

    # 더 하위 섹션 패턴 ((가), (나) 형식)
    sub_sub_pattern = r"^\s*\(([\uac00-\ud7a3])\)\s+(.+?)$"

    # 각 줄을 분석하여 섹션 구조 파악
    lines = content.split("\n")

    # 모든 섹션을 저장할 리스트
    all_sections = []

    # 현재 활성화된 섹션 레벨 추적
    active_sections = {
        1: None,
        2: None,
        3: None,
        4: None,
    }  # 레벨 4는 (1), (2) 형식, 레벨 5는 (가), (나) 형식

    # 임시 섹션 버퍼
    current_section = None
    section_content = []

    # 파일의 각 줄 처리
    for i, line in enumerate(lines):
        line = line.rstrip()

        # 주요 섹션 패턴 확인 (1., 1.1, 1.1.1 형식)
        main_match = re.match(main_pattern, line)
        if main_match:
            # 이전 섹션이 있으면 저장
            if current_section:
                current_section["content"] = "\n".join(section_content).strip()
                all_sections.append(current_section)
                section_content = []

            section_number = main_match.group(1)
            section_title = main_match.group(2)

            # 섹션 레벨 결정
            if section_number.endswith("."):  # 1. 형태
                level = 1
            else:  # 1.1 또는 1.1.1 형태
                level = section_number.count(".") + 1

            # 현재 활성화된 레벨 업데이트
            active_sections[level] = {"number": section_number, "title": section_title}

            # 하위 레벨 초기화
            for l in range(level + 1, 6):
                active_sections[l] = None

            # high_level 정보 생성
            high_level = {}
            for l in range(1, level):
                if active_sections[l]:
                    high_level[f"level_{l}_number"] = active_sections[l]["number"]
                    high_level[f"level_{l}_title"] = active_sections[l]["title"]

            # 새 섹션 시작
            current_section = {
                "type": "main",
                "level": level,
                "number": section_number,
                "title": section_title,
                "high_level": high_level,
                "file_path": file_path,
            }

            # 섹션 내용 시작
            section_content = [line]

        # 하위 섹션 패턴 확인 ((1), (2) 형식)
        elif re.match(sub_pattern, line):
            sub_match = re.match(sub_pattern, line)

            # 이전 섹션이 있으면 저장
            if current_section:
                current_section["content"] = "\n".join(section_content).strip()
                all_sections.append(current_section)
                section_content = []

            sub_number = sub_match.group(1)
            sub_title = sub_match.group(2)

            # high_level 정보 생성
            high_level = {}
            for l in range(1, 4):
                if active_sections[l]:
                    high_level[f"level_{l}_number"] = active_sections[l]["number"]
                    high_level[f"level_{l}_title"] = active_sections[l]["title"]

            # 현재 활성화된 레벨 업데이트 (레벨 4는 (1), (2) 형식)
            active_sections[4] = {"number": f"({sub_number})", "title": sub_title}

            # 하위 레벨 초기화
            active_sections[5] = None

            # 새 하위 섹션 시작
            current_section = {
                "type": "sub",
                "level": 4,  # (1), (2) 형식은 레벨 4로 간주
                "number": f"({sub_number})",
                "title": sub_title,
                "high_level": high_level,
                "file_path": file_path,
            }

            # 섹션 내용 시작
            section_content = [line]

        # 더 하위 섹션 패턴 확인 ((가), (나) 형식)
        elif re.match(sub_sub_pattern, line):
            sub_sub_match = re.match(sub_sub_pattern, line)

            # 이전 섹션이 있으면 저장
            if current_section:
                current_section["content"] = "\n".join(section_content).strip()
                all_sections.append(current_section)
                section_content = []

            subsub_char = sub_sub_match.group(1)
            subsub_title = sub_sub_match.group(2)

            # high_level 정보 생성
            high_level = {}
            for l in range(1, 5):
                if active_sections[l]:
                    high_level[f"level_{l}_number"] = active_sections[l]["number"]
                    high_level[f"level_{l}_title"] = active_sections[l]["title"]

            # 새 하위 섹션 시작
            current_section = {
                "type": "subsub",
                "level": 5,  # (가), (나) 형식은 레벨 5로 간주
                "number": f"({subsub_char})",
                "title": subsub_title,
                "high_level": high_level,
                "file_path": file_path,
            }

            # 섹션 내용 시작
            section_content = [line]

        else:
            # 현재 섹션에 내용 추가
            if current_section:
                section_content.append(line)
            else:
                # 아직 섹션이 시작되지 않았다면, 파일 시작 부분 내용 처리
                if i == 0:
                    current_section = {
                        "type": "intro",
                        "level": 0,
                        "number": "",
                        "title": "Introduction",
                        "high_level": {},
                        "file_path": file_path,
                    }
                    section_content = [line]

    # 마지막 섹션 처리
    if current_section:
        current_section["content"] = "\n".join(section_content).strip()
        all_sections.append(current_section)

    # 각 섹션에 추가 메타데이터 처리
    for section in all_sections:
        # 섹션 식별자 생성
        if section["type"] == "main":
            if section["level"] == 1:  # 최상위 레벨(1.)인 경우
                section["id"] = section["number"]
            else:  # 하위 레벨(1.1, 1.1.1 등)인 경우
                parent_id = ""
                for level in range(1, section["level"]):
                    key = f"level_{level}_number"
                    if key in section["high_level"]:
                        if parent_id:
                            parent_id += "-"
                        parent_id += section["high_level"][key]
                section["id"] = f"{parent_id}-{section['number']}"
        elif section["type"] == "sub":
            parent_id = ""
            for level in [3, 2, 1]:
                key = f"level_{level}_number"
                if key in section["high_level"]:
                    if parent_id:
                        parent_id += "-"
                    parent_id += section["high_level"][key]
                    break
            section["id"] = f"{parent_id}-{section['number']}"
        elif section["type"] == "subsub":
            # 상위 섹션 ID 구성
            parent_ids = []
            # 먼저 메인 섹션(1., 1.1 등) 찾기
            for level in [3, 2, 1]:
                key = f"level_{level}_number"
                if key in section["high_level"]:
                    parent_ids.append(section["high_level"][key])
                    break
            # 그 다음 sub 섹션((1) 등) 찾기
            if "level_4_number" in section["high_level"]:
                parent_ids.append(section["high_level"]["level_4_number"])

            # ID 생성
            section["id"] = "-".join(parent_ids + [section["number"]])
        else:
            section["id"] = "intro"

        # 전체 경로 생성 (예: "1. 개요 > 1.1 배경 > (1) 세부내용 > (가) 세부항목")
        breadcrumb = []

        # 상위 레벨 경로 추가
        for level in [1, 2, 3, 4]:
            key_num = f"level_{level}_number"
            key_title = f"level_{level}_title"
            if key_num in section["high_level"] and key_title in section["high_level"]:
                breadcrumb.append(
                    f"{section['high_level'][key_num]} {section['high_level'][key_title]}"
                )

        # 현재 섹션 추가
        if section["type"] != "intro":
            breadcrumb.append(f"{section['number']} {section['title']}")

        section["breadcrumb"] = " > ".join(breadcrumb)

    return all_sections

# Data Load

In [ ]:
# Directory
data_path = "../data"
pdf_path = f"{data_path}/건설안전지침"  # pdf path


processed_dirname = "documents"  # 전처리 파일 저장
if os.path.exists(f"{data_path}/{processed_dirname}"):
    raise FileExistsError(f"'{processed_dirname}' already exists.")


# PDF
pdf_files = os.listdir(pdf_path)

# PDF 페이지 제거

- 1-2 또는 1-3 페이지 제거 (표지, 목차 등 제거)

In [ ]:
other_file_names = [
    "건설공사 굴착면 안전기울기 기준에 관한 기술지침",
    "건설현장의 중량물 취급 작업계획서(이동식크레인) 작성지침",
    "굴착공사 안전작업 지침",
    "발파공사 안전보건작업 지침",
    "작업의자형 달비계 안전작업 지침",
    "콘크리트공사의 안전보건작업 지침",
    "해체공사 안전보건작업 기술지침",
]

input_directory = pdf_path
output_directory = "./pdf-page-remove"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_pdf_files(input_directory, output_directory, other_file_names)

# PDF 페이지 분리

- olmOCR에서 사용하기 위함

In [ ]:
input_path = "./pdf-page-remove"
output_path = "./pdf-extract-pages"
if not os.path.exists(output_path):
    os.makedirs(output_path)

pdf_files = [f for f in os.listdir(input_path) if f.endswith(".pdf")]

# 자릿수 계산 (PDF 파일이 100개 이상이면 3자리, 아니면 2자리)
pdf_digits = 3 if len(pdf_files) >= 100 else 2

# "pdf_NNN": "원본 파일명" 형식의 딕셔너리 생성
pdf_original_names = {}

for i, pdf_file in enumerate(pdf_files, 1):
    if pdf_file.endswith(".pdf"):
        src_path = f"{input_path}/{pdf_file}"
        doc = fitz.open(src_path)

        # PDF 폴더 이름 생성 (동적 자릿수 사용)
        pdf_folder_format = f"pdf_{i:0{pdf_digits}d}"
        pdf_pages_dir = f"{output_path}/{pdf_folder_format}"

        # 딕셔너리에 원본 파일명 저장
        pdf_original_names[pdf_folder_format] = pdf_file
        if not os.path.exists(pdf_pages_dir):
            os.makedirs(pdf_pages_dir)
        page_count = len(doc)

        # 페이지 자릿수 계산 (페이지가 100개 이상이면 3자리, 아니면 2자리)
        page_digits = 3 if page_count >= 100 else 2

        for page_num in range(page_count):
            new_doc = fitz.open()
            new_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)

            # 페이지 파일 이름 생성 (동적 자릿수 사용)
            page_file_format = f"page_{page_num+1:0{page_digits}d}.pdf"
            page_path = f"{pdf_pages_dir}/{page_file_format}"

            new_doc.save(page_path)
            new_doc.close()

        doc.close()

# olmOCR 로 PDF 텍스트 추출

- `./pdf-test-olmocr.sh` 실행

- `./data-olmocr` 경로에 결과 저장

- 자세한 사용 방법은 [allenai/olmocr](https://github.com/allenai/olmocr) 참고

```python
# Runpod 사양
# 1 x A40
# 9 vCPU 50 GB RAM
# 70 GB Disk  20 GB Pod Volume
# Volume Path: /workspace
# runpod/pytorch:2.4.0-py3.11-cuda12.4.1-devel-ubuntu22.04
```

```bash
PDF_DIR="./pdf-extract-pages"
OUTPUT_DIR="./data-olmocr"

# pdf_001부터 pdf_104까지의 폴더 순차적으로 처리
for i in $(seq -f "%03g" 1 104); do
    folder="pdf_$i"
    echo "폴더 처리 중: $folder"
    
    # 폴더가 존재하는지 확인
    if [ -d "$PDF_DIR/$folder" ]; then
        # 출력 디렉토리 생성
        mkdir -p "$OUTPUT_DIR/$folder"
        
        # olmocr 파이프라인 실행
        echo "처리 중: $PDF_DIR/$folder"
        python -m olmocr.pipeline "$OUTPUT_DIR/$folder" --pdfs "$PDF_DIR/$folder"/*.pdf
    else
        echo "경고: $PDF_DIR/$folder 폴더가 존재하지 않습니다."
    fi
done
```

## 디코딩 및 md 파일로 변환

- `./data-olmocr` 후처리

In [ ]:
# 페이지 수를 저장할 딕셔너리
pdf_page_counts = {}

# 각 PDF 폴더의 페이지 수 계산
for i in range(1, 105):
    pdf_folder = "./pdf-extract-pages/pdf_{i:03d}"
    if os.path.exists(pdf_folder):
        # PDF 파일 개수 세기
        pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))
        pdf_page_counts[f"pdf_{i:03d}"] = len(pdf_files)
    else:
        print(f"pdf_{i:03d}: 폴더 없음")

# 결과 기록용 딕셔너리
results = {}

# JSONL 출력 디렉토리
jsonl_output_dir = "./data-olmocr-decoded/jsonl-files"
if not os.path.exists(jsonl_output_dir):
    os.makedirs(jsonl_output_dir)

# 마크다운 출력 디렉토리
md_output_dir = "./data-olmocr-decoded/md-files"
if not os.path.exists(md_output_dir):
    os.makedirs(md_output_dir)

# 이미 처리된 파일 목록 확인
existing_jsonls = {
    os.path.splitext(os.path.basename(f))[0]
    for f in glob.glob(os.path.join(jsonl_output_dir, "*.jsonl"))
}
existing_mds = {
    os.path.splitext(os.path.basename(f))[0]
    for f in glob.glob(os.path.join(md_output_dir, "*.md"))
}

# pdf_001 부터 pdf_104 까지 반복
for i in range(1, 105):
    pdf_dir = "./data-olmocr/pdf_{i:03d}/results"
    pdf_key = f"pdf_{i:03d}"

    # 이미 처리된 파일인지 확인
    if pdf_key in existing_jsonls and pdf_key in existing_mds:
        results[pdf_key] = "이미 처리됨"
        continue

    if not os.path.exists(pdf_dir):
        results[pdf_key] = "결과 폴더 없음"
        continue

    try:
        # results 디렉토리 내의 모든 jsonl 파일 찾기
        jsonl_files = glob.glob(os.path.join(pdf_dir, "*.jsonl"))

        if not jsonl_files:
            results[pdf_key] = "JSONL 파일 없음"
            continue

        for jsonl_file in jsonl_files:
            data = []
            with open(jsonl_file, "r", encoding="utf-8-sig") as file:
                for line in file:
                    if line.strip():
                        data.append(json.loads(line))

            # 소스 파일 경로에서 pdf_XXX 부분 추출하여 파일명 생성
            if data and "metadata" in data[0] and "Source-File" in data[0]["metadata"]:
                source_file_path = data[0]["metadata"]["Source-File"]
                pdf_folder_name = os.path.basename(os.path.dirname(source_file_path))
                base_filename = pdf_folder_name
            else:
                # 메타데이터가 없는 경우 원본 폴더명 사용
                base_filename = pdf_key

            # 이미 처리된 파일인지 다시 확인 (폴더명 기준)
            if base_filename in existing_jsonls and base_filename in existing_mds:
                results[base_filename] = "이미 처리됨"
                continue

            # 데이터를 jsonl 파일로 저장
            if data:
                # JSONL 파일 생성
                jsonl_output_path = os.path.join(
                    jsonl_output_dir, f"{base_filename}.jsonl"
                )
                with open(jsonl_output_path, "w", encoding="utf-8-sig") as file:
                    for item in data:
                        json_line = json.dumps(item, ensure_ascii=False)
                        file.write(json_line + "\n")

                # 마크다운 파일 생성
                md_output_path = os.path.join(md_output_dir, f"{base_filename}.md")
                with open(md_output_path, "w", encoding="utf-8-sig") as md_file:
                    # 모든 페이지의 텍스트를 마크다운에 추가
                    for page_num, item in enumerate(data, 1):
                        if "text" in item:
                            # 텍스트 추가
                            md_file.write(f"{item['text']}\n\n")

                # PDF 페이지 수와 JSONL 라인 수 비교
                expected_pages = pdf_page_counts.get(base_filename, 0)
                if expected_pages > 0:
                    if len(data) == expected_pages:
                        results[base_filename] = "일치"
                    else:
                        results[base_filename] = (
                            f"불일치 (JSONL: {len(data)}, PDF: {expected_pages})"
                        )
                else:
                    results[base_filename] = f"페이지 수 비교 불가 (JSONL: {len(data)})"
            else:
                results[pdf_key] = "데이터 없음"
    except Exception as e:
        results[pdf_key] = f"오류: {str(e)}"

# 결과 출력
for pdf_key in sorted(results.keys()):
    print(f"{pdf_key}: {results[pdf_key]}")

## 파일명 변경

- `pdf_NNN` 을 원본 파일명으로 변경

In [ ]:
# 원본 파일명 리스트
org_file_name_list = sorted(os.listdir("./pdf-page-remove"))
jsonl_file_name_list = sorted(
    os.listdir("./data-olmocr-decoded/jsonl-files")
)
md_file_name_list = sorted(os.listdir("./data-olmocr-decoded/md-files"))

# JSONL 파일 이름 변경
jsonl_count = 0
for jsonl_file in jsonl_file_name_list:
    if jsonl_file.startswith("pdf_"):
        # pdf_001 형식에서 숫자 부분 추출
        pdf_key = jsonl_file.split(".")[0]  # 확장자 제거

        if pdf_key in pdf_original_names:
            # 원본 파일명 가져오기
            org_name = pdf_original_names[pdf_key]

            # JSONL 파일 이름 변경
            old_jsonl_path = (
                "./data-olmocr-decoded/jsonl-files/{jsonl_file}"
            )
            new_jsonl_path = (
                "./data-olmocr-decoded/jsonl-files/{org_name}.jsonl"
            )

            if os.path.exists(old_jsonl_path) and not os.path.exists(new_jsonl_path):
                os.rename(old_jsonl_path, new_jsonl_path)
                jsonl_count += 1

# 마크다운 파일 이름 변경
md_count = 0
for md_file in md_file_name_list:
    if md_file.startswith("pdf_"):
        # pdf_001 형식에서 숫자 부분 추출
        pdf_key = md_file.split(".")[0]  # 확장자 제거

        if pdf_key in pdf_original_names:
            # 원본 파일명 가져오기
            org_name = pdf_original_names[pdf_key]

            # MD 파일 이름 변경
            old_md_path = "./data-olmocr-decoded/md-files/{md_file}"
            new_md_path = "./data-olmocr-decoded/md-files/{org_name}.md"

            if os.path.exists(old_md_path) and not os.path.exists(new_md_path):
                os.rename(old_md_path, new_md_path)
                md_count += 1

print(f"JSONL 파일 {jsonl_count}개, MD 파일 {md_count}개의 이름이 변경되었습니다.")

## md_files 분할 후 jsonl 형식으로 저장

In [ ]:
md_files_path = "./data-olmocr-decoded/md-files"
md_files_list = [
    f"{md_files_path}/{file}"
    for file in os.listdir(md_files_path)
    if file.endswith(".md")
]

jsonl_output_path = f"{data_path}/{processed_dirname}"  # ../data/documents
if not os.path.exists(jsonl_output_path):
    os.makedirs(jsonl_output_path)

# 각 파일의 섹션을 추출하여 JSONL로 저장
for file in md_files_list:
    file_name = os.path.basename(file)

    # 파일을 섹션으로 분할
    sections = split_document_for_md(file, remove_figures=True)

    # JSONL 파일로 저장
    jsonl_file_path = f"{jsonl_output_path}/{file_name.replace('.md', '.jsonl')}"

    with open(jsonl_file_path, "w", encoding="utf-8-sig") as f:
        for section in sections:
            # Document 형식으로 변환 (page_content와 metadata)
            document = {
                "page_content": section["content"],  # 섹션 내용
                "metadata": {
                    "id": section["id"],
                    "type": section["type"],
                    "number": section["number"],
                    "level": section["level"],
                    "title": section["title"],
                    "breadcrumb": section["breadcrumb"],
                    "high_level": section["high_level"],
                    "file_path": section["file_path"],
                    "file_name": file_name.replace(".md", ""),
                },
            }
            f.write(json.dumps(document, ensure_ascii=False) + "\n")